## 前処理したデータをSageMaker用とローカル用とに作り分ける

### SageMakerの処理
#### 前提：教師データ：学習用
- 目的変数　Y_train.csvにある
- 説明変数　X_train.csvにある

#### 前提：教師データ：検証用
- 目的変数　Y_eval.csv
- 説明変数　X_eval.csv

- データはノートブックから直接read_csvできるところにある。
- 教師データは、一列目が目的変数である必要がある。ロジックでこのデータを組み立てること